In [1]:
"""
批量读取.csv特征集(bicps or tricps)
对临床数据healthy和affected两侧分别进行每个人的单独归一化
"""
import pandas as pd

from utils_iFEMG_feature import df_save_csv, df_norm, df_merge_colname, df_norm_clinical

In [ ]:
# healthy curl 0-2kg 完整数据集名单
'''
0912s1, 1017s2: 0-1kg
0926s2 只包含了2kg和伸展0.5kg最大值
'''
['w8s1', '0912s2', '0919s2', '0926s1', '1010s1', '1017s1', '1024s1', '1107s1']

In [ ]:
# healthy extension 0-1kg 完整数据集名单
'''
只有0kg: 0912s1(患侧<0), 0912s2, 0926s1(患侧<0), 
0 - 0.5: 0926s2(健侧只有0.5), '1010s1',
'''
['0919s2']

In [6]:
"""
获取文件路径下多个文件夹中.csv格式的特征集

1. 文件夹路径
2. 肌肉bi tri
"""
# 文件夹路径 健康人数据
folder_path = r"E:\Data\paper2\积水潭患者数据集"

# subject_name_list = ['w8s1', '0912s1', '0912s2', '0919s2', '0926s1', '0926s2', '1010s1', '1017s1', '1017s2', '1024s1', '1107s1']    # affected 患侧包含0-1kg完整数据集
subject_name_list = ['w8s1', '0912s1', '0912s2', '0919s2', '0926s1', '1010s1', '1017s1', '1017s2', '1024s1', '1107s1']  # healthy 健侧包含0-1kg完整数据集

side = 'healthy'    # healthy or affected

file_df_list = []
for name in subject_name_list:
    file_path = folder_path + '\\' + name + '\\' + side + '\\' + name + "_curl_abs.csv"
    try:
        file_df_list.append(pd.read_csv(file_path, header = [0,1], index_col = None))
    except FileNotFoundError:
        print(f"文件 {name} 未读取")
    pass

print(f"文件数: {len(file_df_list)}")


文件数: 10


In [7]:
"""
对每名被试信号特征分别单独归一化

NOTE:
    - 需要修改保存文件的文件名！！
"""

# 归一化某特征
normed_df_list = [] 
err_df_index = 0    # 如果有key error发生报错，报告所在df的index
# 需要进行归一化的特征列名
col_name2sacle = [("bicps_br", "initial_pressure_ave"),
                    ("bicps_br", "FMG"),
                    ("bicps_br", "mav"),
                    ("bicps_br", "rms"),
                    ("bicps_br", "wave_length"),
                    ("bicps_br", "zero_crossing"),
                    ("bicps_br", "slope_sign_change"),
                    ("bicps_br", "mean_freq"),
                    ("bicps_br", "mean_power_freq"),
                    ("tricps_br_medial", "initial_pressure_ave"),
                    ("tricps_br_medial", "FMG"),
                    ("tricps_br_medial", "mav"),
                    ("tricps_br_medial", "rms"),
                    ("tricps_br_medial", "wave_length"),
                    ("tricps_br_medial", "zero_crossing"),
                    ("tricps_br_medial", "slope_sign_change"),
                    ("tricps_br_medial", "mean_freq"),
                    ("tricps_br_medial", "mean_power_freq"),
                    ("tricps_br_lateral", "initial_pressure_ave"),
                    ("tricps_br_lateral", "FMG"),
                    ("tricps_br_lateral", "mav"),
                    ("tricps_br_lateral", "rms"),
                    ("tricps_br_lateral", "wave_length"),
                    ("tricps_br_lateral", "zero_crossing"),
                    ("tricps_br_lateral", "slope_sign_change"),
                    ("tricps_br_lateral", "mean_freq"),
                    ("tricps_br_lateral", "mean_power_freq")]

for df in file_df_list:
    # 对指定的列进行归一化
    try:
        normed_df_list.append(df_norm_clinical(dataframe=df,
                                               col_name=col_name2sacle,
                                               method='z_score',
                                               train_col_name=('subject_info', 'label'),
                                               train_col_values=[0, 0.5, 1]))
    except KeyError:
        print(f"index {err_df_index} 存在错误")
        pass
    err_df_index = err_df_index + 1
    pass
# 合并dataframe
final_data_df = pd.concat(normed_df_list, axis = 0, ignore_index = True)

In [8]:
# 合并列名
final_data_df_n = df_merge_colname(final_data_df)

final_data_df_n

,subject_info_subject_name,subject_info_height,subject_info_weight,subject_info_age,subject_info_gender,subject_info_label,bicps_br_initial_pressure_ave,bicps_br_FMG,bicps_br_mav,bicps_br_rms,...,tricps_br_medial_mean_power_freq,tricps_br_lateral_initial_pressure_ave,tricps_br_lateral_FMG,tricps_br_lateral_mav,tricps_br_lateral_rms,tricps_br_lateral_wave_length,tricps_br_lateral_zero_crossing,tricps_br_lateral_slope_sign_change,tricps_br_lateral_mean_freq,tricps_br_lateral_mean_power_freq
0,w8s1,163,77.5,73,0,0.0,-1.423359,-1.291916,1.971212,2.108249,...,-5.766996,-0.934450,-1.191697,-0.183983,-0.186029,-0.306509,-0.142589,0.829742,-1.050480,-0.688240
1,w8s1,163,77.5,73,0,0.0,-1.423359,-1.580799,1.190966,1.168266,...,0.028100,-0.934450,-1.113875,-0.685712,-0.492337,-0.890581,-0.209159,1.099840,-0.036222,0.172613
2,w8s1,163,77.5,73,0,0.0,-1.423359,-1.341296,1.210129,1.207858,...,0.535397,-0.934450,-0.985677,-0.570882,-0.416656,-0.879198,-0.839052,0.716029,-0.772133,-0.214019
3,w8s1,163,77.5,73,0,0.0,-1.423359,-1.452192,0.920026,0.859304,...,0.176962,-0.934450,-0.943899,-0.808321,-0.570452,-1.110686,-0.432115,0.974747,-0.889768,-0.493292
4,w8s1,163,77.5,73,0,0.0,-1.423359,-1.453546,0.919553,0.883412,...,0.591925,-0.934450,-0.857876,-0.863787,-0.615785,-1.154339,-0.088140,1.285148,-0.566615,-0.089723
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
300,1107s1,150,60.0,66,0,2.0,-0.152116,1.745725,-0.291616,-0.312781,...,-1.474469,-1.326712,0.075108,-0.174035,-0.174041,-0.173923,-0.051001,0.135446,0.005447,0.163502
301,1107s1,150,60.0,66,0,2.0,-0.152116,2.143042,-0.295205,-0.331501,...,-0.607523,-1.326712,-1.670093,-0.174017,-0.174043,-0.174245,0.014861,-0.080269,0.024198,0.043082
302,1107s1,150,60.0,66,0,2.0,-0.152116,1.909581,0.575382,1.139927,...,1.355713,-1.326712,0.095733,-0.174143,-0.174119,-0.174838,-0.085516,0.112657,0.320786,0.082729
303,1107s1,150,60.0,66,0,2.0,-0.152116,2.236475,-0.325482,-0.423538,...,-0.235816,-1.326712,0.177455,-0.174033,-0.174044,-0.173137,0.247804,0.382589,0.277229,0.112444


In [9]:
# 保存文件
df_save_csv(final_data_df_n, r"E:\Data\paper2\积水潭患者数据集\iFEMG_curl_abs_normed_healthy.csv")

File E:\Data\paper2\积水潭患者数据集\iFEMG_curl_abs_normed_healthy.csv saved!
